In [1]:
import csv
import pandas as pd
import numpy as np
import glob
import matplotlib
import scipy as sp
from matplotlib import pyplot as plt
from itertools import cycle, islice
from scipy import stats

import seaborn as sns

In [2]:
def organize_trial_by_trial(txt_filename):
    reader = csv.reader( open(txt_filename))
    game_data = [row for row in reader]
    df = pd.DataFrame(np.array(game_data))
    df = df.T
    df = df.rename(columns = {0:"game number", 1:"trial", 2:"player", 3:"arm", 4:"probability", 5:"reward", 6:"time", 7:"P1 score", 8:"P2 score"})
    #df = df[40:]
    #df = df.drop(columns = 'time') #can comment this out if want to look at time
    df = df.apply(pd.to_numeric)
    df.head()
    
    #combining info in the same trial for both players together in the same row
    #player1 = df[df['player'] == 1].drop(columns = "P2 score").drop(columns = "player")
    #player1 = player1.reset_index().drop(columns = "index")
    #player1 = player1.rename(columns = {"arm":"P1 arm", "probability":"P1 prob", "reward":"P1 reward"})

    #player2 = df[df['player'] == 2].drop(columns = "P1 score").drop(columns = 'trial').drop(columns = 'player')
    #player2 = player2.reset_index().drop(columns = "index")
    #player2 = player2.rename(columns = {"arm":"P2 arm", "probability":"P2 prob", "reward":"P2 reward"})

    #result = pd.concat([player1, player2], axis=1, sort=False)
    return df

#TODO: function that pairs up P2(trial n) and P1(trial n+1)


#def combine_all_files(path):
    #all_files = glob.glob(path + "/*.txt")
    #df = pd.DataFrame()
    
    #for filename in all_files:
     #   new_df = organize_trial_by_trial(filename)
      #  df = pd.concat([df, new_df], axis=0, ignore_index=True)
    #print(df)
    #return df%

In [3]:
def get_score(df, player_number, game_number):
    if game_number == 1:
        row = 199
    elif game_number == 2:
        row = 399
    if player_number == 1:
        column = 'P1 score'
    elif player_number == 2:
        column = 'P2 score'
    score = df.loc[row, column]
    return score

In [4]:
#create a dataframes for scores
scores = pd.DataFrame(columns=['s_g1','s_g2','j_g1','j_g2'])

In [5]:
#version is 's' for single player and 'j' for joint
def call_get_score(df, version):
    p1_g1 = get_score(df, 1, 1)
    p1_g2 = get_score(df, 1, 2)
    p1_score_diff = p1_g2 - p1_g1
    if version == 'j':
        p2_g1 = get_score(df, 2, 1)
        p2_g2 = get_score(df, 2, 2)
        p2_score_diff = p2_g2 - p2_g1
        return p1_g1, p1_g2, p2_g1, p2_g2, p1_score_diff, p2_score_diff
    return p1_g1, p1_g2, p1_score_diff

In [6]:
#separate data frames for p1 and p2 
def separate_df(df, version):
    #df.set_index(df['trial'])
    df_p1 = df[df['player'] == 1]
    df_p1_g1 = df_p1[df['game number'] == 1]
    df_p1_g2 = df_p1[df['game number'] == 2]
    #remove first 20 trials for g2
    #df_p1_g2 = df_p1_g2[20:]
    if version == 's':
        return df_p1_g1, df_p1_g2
    df_p2 = df[df['player'] ==2 ]
    df_p2_g1 = df_p2[df['game number'] == 1]
    df_p2_g2 = df_p2[df['game number'] == 2]
    return df_p1_g1, df_p1_g2, df_p2_g1, df_p2_g2

In [7]:
#needs work
def call_call_get_score(df, version):
    p1_g1 = df+'_'+'p1_'+'g1_'+'score'
    p1_g2 = df+'_'+'p1_'+'g2_'+'score'
    p2_g1 = df+'_'+'p2_'+'g1_'+'score'
    p2_g2 = df+'_'+'p2_'+'g2_'+'score'
    p1_g1, p1_g2, p2_g1, p2_g2 = call_get_score(df, version)
    return p1_g1, p1_g2, p2_g1, p2_g2

In [12]:
#in the s files, p2 is the confederate
df = organize_trial_by_trial('data/s01_gamedata.csv')
c04 = df
c04_p1_g1, c04_p1_g2 = separate_df(c04,'s')
c04_p1_g1_score,c04_p1_g2_score,c04_score_diff = call_get_score(c04, 's')
print(c04_p1_g1_score,c04_p1_g2_score,c04_score_diff)
c04.head()
c04_p1_g1_count = c04_p1_g1['probability'].value_counts(sort=False)
c04_p1_g2_count = c04_p1_g2['probability'].value_counts(sort=False)

60 74 14


/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/baryosef/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [13]:
def jsd(p, q, base=np.e):
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    ## convert to np.array
    p, q = np.asarray(p), np.asarray(q)
    ## normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return sp.stats.entropy(p,m, base=base)/2. +  sp.stats.entropy(q, m, base=base)/2.

In [14]:
def compare_strat(p1,p2):
    strat = jsd(p1,p2)
    comp_1 = jsd(p1,[j_g1_low,j_g1_med,j_g1_high])
    comp_2 = jsd(p2,[j_g1_low,j_g1_med,j_g1_high])
    return strat, comp_1, comp_2

In [15]:
maximize = [0,0,100]
match = [30/180 * 100,60/180 * 100,50]

In [16]:
def call_jsd(counts):
    maxi = jsd(counts,maximize)
    matchi = jsd(counts,match)
    return maxi,matchi

In [17]:
c04_g1_max,c04_g1_match = call_jsd(c04_p1_g1_count)
c04_g2_max,c04_g2_match = call_jsd(c04_p1_g2_count)

In [18]:
c04_scores = pd.DataFrame({'g1_score':[c04_p1_g1_score],'g2_score':c04_p1_g2_score,'score_diff':c04_score_diff})

In [19]:
c04_scores

,g1_score,g2_score,score_diff
0,60,74,14


In [20]:
c04_p1_g1_count

30    38
60    43
90    19
Name: probability, dtype: int64

In [21]:
c04_p1_g2_count

30    38
60    16
90    46
Name: probability, dtype: int64

In [22]:
print(c04_g1_max,c04_g1_match,c04_g2_max,c04_g2_match)

0.4318754981734429 0.06054148947299787 0.23828696089939724 0.03734820425576257


In [24]:
c04_p1_g1

,game number,trial,player,arm,probability,reward,time,P1 score,P2 score
0,1,1,1,2,30,1,11.220698,1,0
2,1,2,1,3,60,1,5.362458,2,1
4,1,3,1,2,30,0,3.489552,2,2
6,1,4,1,3,60,0,3.210886,2,3
8,1,5,1,3,60,1,2.904784,3,4
10,1,6,1,1,90,0,3.241885,3,5
12,1,7,1,1,90,1,2.839384,4,6
14,1,8,1,2,30,0,2.985595,4,7
16,1,9,1,2,30,1,2.581026,5,8
18,1,10,1,2,30,0,2.472073,5,9


In [26]:
c04_p1_g2

,game number,trial,player,arm,probability,reward,time,P1 score,P2 score
200,2,1,1,2,90,1,4.426063,1,0
202,2,2,1,3,30,1,2.593545,2,0
204,2,3,1,1,60,0,2.968871,2,1
206,2,4,1,2,90,1,2.554801,3,2
208,2,5,1,2,90,1,2.727741,4,3
210,2,6,1,3,30,0,1.938397,4,4
212,2,7,1,2,90,1,2.695570,5,4
214,2,8,1,2,90,1,2.809592,6,5
216,2,9,1,2,90,1,3.375869,7,6
218,2,10,1,3,30,0,2.936245,7,7
